In [1]:
import cvxpy as cp
import numpy as np

from compas_pr2d import arch
from compas_pr2d.prd import PR2DModel

height = 4
span = 10
n = 4
thickness = 0.5

print("Constructing the model ...")
model = PR2DModel()
print("Constructing the arch geometry ...")
arch_geometry = arch.construct_arch(height, span, n, thickness)
print("Feeding the geometry to the model ...")

model.from_polygons(arch_geometry)
# model.display_mesh(bindex=True, nindex=True)

print("Assembling Matrices:")

model.set_boundary_edges(display=False)
disp_data = [
    [4, (-0.1,-0.0), (-0.1,-0.1)],
]  # [Interface ID, normal displacement, tangential displacement]


# model.set_force()  # for now no loads, just self-weight which is default in the model (i will add as gravity to all blocks for now only)
print("Assigning BCs to RHS vectors...")
model.assign_bc(disp_data)
# print("Solving the system...")
model.solve()
model.display_results()
deformed = model.results.deformed_shape



Constructing the model ...
Constructing the arch geometry ...
Feeding the geometry to the model ...
Assembling Matrices:
Assembling boundary elements for edge: (0, 3, (0, 1))
Boundary edge rows: 6, 7
Assembling boundary elements for edge: (3, 4, (8, 9))
Boundary edge rows: 8, 9
Assigning BCs to RHS vectors...
Assigning displacement BC at interface 4 on edge (8, 9)
[ 0.00331136  0.00270369 -0.00227778  0.00808914  0.00867591 -0.00227778
 -0.00635884  0.04523792 -0.0195     -0.04726128  0.09636597 -0.0195    ]


In [2]:

A_ub = model.A_ub
A_eq = model.A_eq
U    = np.asarray(model.results.U).reshape(-1)
c    = np.asarray(model.c).reshape(-1)

m_ub, _ = A_ub.shape
m_eq, _ = A_eq.shape

# Stacking the dual problem data as in the antonino's paper
# As the CVXPY solver expects the objective function to be a single vector
# not a decoupled fn and ft.
delta_n = -model.b_ub
delta_t = -model.b_eq
# So all the data will be horitzontally stacked as follows:
# delta = np.hstack([A_ub @ U, A_eq @ U])          # (m_ub + m_eq,)
A_T    = np.hstack([A_ub.T,  A_eq.T])            # (n, m_ub + m_eq)

delta = np.hstack([delta_n, delta_t])            # (m_ub + m_eq,)
f = cp.Variable(m_ub + m_eq)

mu = 0.3
constraints = [
    A_T @ f == c,
    f[:m_ub] <= 0,
]
fn = f[:m_ub]
ft = f[m_ub:]

constraints += [
    ft <=  mu * (-fn),
    ft >= -mu * (-fn),
]
objective = cp.Minimize(-(delta @ f))

prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.MOSEK)
print("status:", prob.status)
print("objective:", prob.value)

f_opt = f.value                   

# Retrieveing the normal and tangential components of the force vector
fn_opt = f_opt[:m_ub]                # normal component
ft_opt = f_opt[m_ub:]                # tangential component

# Verify the optimality condition after decomposing the force vector.
res = A_ub.T @ fn_opt + A_eq.T @ ft_opt - c
print("||res||2 =", np.linalg.norm(res))


status: optimal
objective: -1.4965004124210932
||res||2 = 3.583538750481339e-11


In [3]:
print("fn:", fn_opt.reshape(-1,2))
print("ft:", ft_opt.reshape(-1,2))
print("fn shape:", fn_opt.shape, "ft shape:", ft_opt.shape)


edges = model.all_contacts
i = len(edges)
nodes = model.n

for edge in edges:
    print(f"Edge {i}: {edge} connects nodes {edge[0]} and {edge[1]}")
    i -= 1


fn: [[-4.57641656e+00 -1.02089987e+01]
 [-1.09500030e+01 -1.54896540e-10]
 [-6.76255747e+00 -7.80957574e+00]
 [-3.74726910e+00 -1.79643988e+01]
 [-2.13785773e+01 -4.79439872e-12]]
ft: [[-4.85560780e-01 -4.67650860e-01]
 [ 1.70708926e-01 -1.27388898e-11]
 [ 1.90448214e-01  4.96160876e-01]
 [ 1.03181078e+00  5.30681703e+00]
 [ 6.41357320e+00  4.92055846e-12]]
fn shape: (10,) ft shape: (10,)
Edge 5: (0, 1, 0, (2, 3)) connects nodes 0 and 1
Edge 4: (1, 2, 1, (4, 5)) connects nodes 1 and 2
Edge 3: (2, 3, 2, (6, 7)) connects nodes 2 and 3
Edge 2: (0, 3, (0, 1)) connects nodes 0 and 3
Edge 1: (3, 4, (8, 9)) connects nodes 3 and 4


In [4]:
import compas.geometry as cg
from compas.colors import Color
from compas_viewer import Viewer
from compas_viewer.scene import Tag


def weighted_point_on_segment(p0, p1, w0, w1):
    wsum = w0 + w1
    if wsum == 0:
        return cg.Point(0.5 * (p0.x + p1.x), 0.5 * (p0.y + p1.y), 0.5 * (p0.z + p1.z))
    return cg.Point(
        (w0 * p0.x + w1 * p1.x) / wsum,
        (w0 * p0.y + w1 * p1.y) / wsum,
        (w0 * p0.z + w1 * p1.z) / wsum,
    )

edges = model.all_contacts
nodes = model.n

points_t = []
viewer = Viewer()
# for shape in deformed:
#     viewer.scene.add(shape)
viewer.scene.add(model.mesh,opacity=0.5)
scale = 0.01
print(len(edges))
for bid,e in enumerate(edges):
    if len(e) == 4:
        i, j, k, (u, v) = e
    elif len(e) == 3:
        i, k, (u, v) = e
    else:
        continue

    A = cg.Point(*nodes[u])
    B = cg.Point(*nodes[v])
    line = cg.Line(A, B)

    # Get the normal force values at the endpoints of the edge

    fnA = float(fn_opt[2*k])   # end at node A
    fnB = float(fn_opt[2*k+1])   # end at node B

    # Weighted application point
    wA, wB = abs(fnA), abs(fnB)
    N = fnA + fnB
    pc = weighted_point_on_segment(A, B, wA, wB)

    t = cg.Vector(*line.vector)
    t.y = 0.0
    if t.length == 0:
        continue
    t.unitize()

    n = cg.Vector(-t.z, 0.0, t.x)
    if n.length == 0:
        continue
    n.unitize()

    # Magnitude (use average or resultant; here average)
    # tip = cg.Point(pc.x, pc.y, pc.z)
    # tip.translate(n * (scale * N))
    points_t.append([pc, N])
    # viewer.scene.add(cg.Line(pc, tip))   # <-- NO anchor here
    t = Tag(k.__str__(),position=pc)
    # viewer.scene.add(t)

points, N = zip(*points_t)
points = list(points)
polyline = cg.Polyline(points)

for point, n in zip(points, N):
    viewer.scene.add(point, vertexcolor=Color.from_number(n), pointsize=25 )
# viewer.scene.add(polyline)
viewer.show()


5
